Lista 5 Zadanie 1

In [72]:
import numpy as np
import random

In [73]:
def safe_add(x, y):
    return np.clip(x + y, -1e6, 1e6)

def safe_sub(x, y):
    return np.clip(x - y, -1e6, 1e6)

def safe_mul(x, y):
    return np.clip(x * y, -1e6, 1e6)

def safe_div(x, y):
    with np.errstate(divide='ignore', invalid='ignore'):
        result = np.true_divide(x, y)
        result[~np.isfinite(result)] = 0
    return np.clip(result, -1e6, 1e6)

In [74]:
FUNCTIONS = [safe_add, safe_sub, safe_mul, safe_div]
FUNCTION_NAMES = ['+', '-', '*', '/']

In [75]:
class Node:
    def __init__(self, function=None, left=None, right=None, value=None):
        self.function = function
        self.left = left
        self.right = right 
        self.value = value

    def evaluate(self, X):
        if self.value is not None:
            return X[:, self.value]
        left_val = self.left.evaluate(X)
        right_val = self.right.evaluate(X)
        try:
            return self.function(left_val, right_val)
        except ZeroDivisionError:
            return np.zeros_like(left_val) 

    def __str__(self):
        if self.value is not None:
            return f"x[{self.value}]"
        return f"({self.left} {FUNCTION_NAMES[FUNCTIONS.index(self.function)]} {self.right})"

def generate_random_tree(depth, num_features):
    if depth == 0 or (depth > 1 and random.random() < 0.3):
        return Node(value=random.randint(0, num_features - 1))
    function = random.choice(FUNCTIONS)
    return Node(function=function,
                left=generate_random_tree(depth-1, num_features),
                right=generate_random_tree(depth-1, num_features))

def fitness(tree, X, y):
    y_pred = tree.evaluate(X)
    mse = np.mean((y - y_pred) ** 2)
    return mse

def crossover(parent1, parent2, num_features):
    if parent1 is None or parent2 is None:
        return parent1 if parent2 is None else parent2
    
    if parent1.value is not None and parent2.value is not None:
        return Node(value=random.choice([parent1.value, parent2.value]))

    new_node = Node(function=random.choice([
        parent1.function if parent1.function else random.choice(FUNCTIONS),
        parent2.function if parent2.function else random.choice(FUNCTIONS)
    ]))

    new_node.left = crossover(parent1.left if parent1.left else Node(value=random.randint(0, num_features - 1)),
                            parent2.left if parent2.left else Node(value=random.randint(0, num_features - 1)), num_features)
    new_node.right = crossover(parent1.right if parent1.right else Node(value=random.randint(0, num_features - 1)),
                            parent2.right if parent2.right else Node(value=random.randint(0, num_features - 1)), num_features)

    return new_node

def mutate(tree: Node, num_features, max_depth):
    if random.random() < 0.4 and tree.function is not None:
        tree.function = random.choice(FUNCTIONS)
    elif random.random() < 0.4 and tree.value is not None:
        tree.value = random.randint(0, num_features - 1)
    else:
        if random.random() < 0.4 and tree.left is not None:
            tree.left = generate_random_tree(max_depth, num_features)
        elif random.random() < 0.4 and tree.right is not None:
            tree.right = generate_random_tree(max_depth, num_features)
        else:
            tree = generate_random_tree(max_depth, num_features)
    return tree

def genetic_programming(X, y, num_generations=10000, population_size=100, max_depth=3):
    population = [generate_random_tree(max_depth, X.shape[1]) for _ in range(population_size)]
    for generation in range(num_generations):
        fitness_scores = np.array([fitness(tree, X, y) for tree in population])
        sorted_indices = np.argsort(fitness_scores)
        sorted_population = [population[i] for i in sorted_indices]

        # print(f"Generacja {generation}, Najlepsze MSE: {min(fitness_scores)}")

        elite_size = 5
        elite_population = sorted_population[:elite_size]
        population = sorted_population[:population_size // 2]
        population = elite_population + population[:population_size - elite_size]
        while len(population) < population_size:
            parent1, parent2 = random.choices(sorted_population[:population_size // 2], k=2)
            child = crossover(parent1, parent2, X.shape[1])
            if random.random() < 0.5:
                child = mutate(child, X.shape[1], max_depth)
            population.append(child)
    return sorted_population[0]

In [76]:
train_file_path = "srsd-feynman_medium/train/feynman-i.11.19.txt"
test_file_path = "srsd-feynman_medium/test/feynman-i.11.19.txt"

train_data = np.loadtxt(train_file_path)
train_X = train_data[:, :-1]
train_y = train_data[:, -1]

test_data = np.loadtxt(test_file_path)
test_X = test_data[:, :-1]
test_y = test_data[:, -1]

In [ ]:
best_tree = genetic_programming(train_X, train_y)
print("\nNajlepsze rozwiązanie:")
print(best_tree)
print(fitness(best_tree, train_X, train_y))
print(fitness(best_tree, test_X, test_y))

print("Oryginalna funkcja:\n x[0]*x[1] + x[2]*x[3]+x[4]*x[5]")



Najlepsze rozwiązanie:
(((x[1] * x[0]) + (x[5] * x[4])) + (((x[5] - x[5]) / (x[3] + x[3])) + ((x[3] * x[2]) - (x[3] - x[3]))))
1.9774141940547198e-30
1.730921400464013e-30
Oryginalna funkcja:
 x[0]*x[1] + x[2]*x[3]+x[4]*x[5]


In [78]:
train_file_path = "srsd-feynman_medium/train/feynman-i.34.8.txt"
test_file_path = "srsd-feynman_medium/test/feynman-i.34.8.txt"

train_data = np.loadtxt(train_file_path)
train_X = train_data[:, :-1]
train_y = train_data[:, -1]

test_data = np.loadtxt(test_file_path)
test_X = test_data[:, :-1]
test_y = test_data[:, -1]


In [79]:
best_tree = genetic_programming(train_X, train_y)
print("\nNajlepsze rozwiązanie:")
print(best_tree)
print(fitness(best_tree, train_X, train_y))
print(fitness(best_tree, test_X, test_y))

print("Oryginalna funkcja:\n (x[0]*x[1]*x[2])/x[3]")



Najlepsze rozwiązanie:
(((x[2] / x[3]) / (x[0] - x[3])) * ((x[0] - x[3]) * (x[0] * x[1])))
7.4052368174492345e-53
5.867977945288164e-53
Oryginalna funkcja:
 (x[0]*x[1]*x[2])/x[3]


In [80]:
train_file_path = "srsd-feynman_easy_dummy/train/feynman-i.12.1.txt"
test_file_path = "srsd-feynman_easy_dummy/test/feynman-i.12.1.txt"

train_data = np.loadtxt(train_file_path)
train_X = train_data[:, :-1]
train_y = train_data[:, -1]

test_data = np.loadtxt(test_file_path)
test_X = test_data[:, :-1]
test_y = test_data[:, -1]


In [81]:
best_tree = genetic_programming(train_X, train_y, num_generations=10000)
print("\nNajlepsze rozwiązanie:")
print(best_tree)
print(fitness(best_tree, train_X, train_y))
print(fitness(best_tree, test_X, test_y))

print("Oryginalna funkcja:\n (x[0]*x[1])")


Najlepsze rozwiązanie:
((((x[0] / x[2]) * (x[2] - x[1])) * ((x[2] * x[0]) * (x[0] / x[2]))) + (((x[2] + x[0]) / (x[1] / x[1])) / ((x[1] / x[1]) / (x[1] - x[2]))))
2.063460999475474e-17
1.5829485265651865e-17
Oryginalna funkcja:
 (x[0]*x[1])


In [82]:
train_file_path = "srsd-feynman_easy_dummy/train/feynman-i.25.13.txt"
test_file_path = "srsd-feynman_easy_dummy/test/feynman-i.25.13.txt"

train_data = np.loadtxt(train_file_path)
train_X = train_data[:, :-1]
train_y = train_data[:, -1]

test_data = np.loadtxt(test_file_path)
test_X = test_data[:, :-1]
test_y = test_data[:, -1]


In [84]:
best_tree = genetic_programming(train_X, train_y, num_generations=10000)
print("\nNajlepsze rozwiązanie:")
print(best_tree)
print(fitness(best_tree, train_X, train_y))
print(fitness(best_tree, test_X, test_y))

print("Oryginalna funkcja:\n (x[0]/x[1])")


Najlepsze rozwiązanie:
((((x[1] * x[1]) - (x[1] + x[1])) / ((x[0] - x[1]) / (x[1] / x[2]))) + (((x[2] - x[0]) / (x[1] - x[0])) / ((x[0] - x[0]) + (x[2] / x[1]))))
2.1043416214478898e-13
2.3692766439142076e-13
Oryginalna funkcja:
 (x[0]/x[1])
